# K-Map Visualizer
This Jupyter notebook allows users to see how a kmap can be solved.

Simply input the matrix you want and press play.

In [55]:
from manim import *

import numpy as np
import kmap_animator
config.media_width = "100%"
_RV = "-v WARNING -qm --progress_bar None --disable_caching Example"

In [56]:
from pyeda.inter import *
from pyeda.inter import And 
import pyeda.boolalg as boolalg

a, b, c, d = map(exprvar, 'abcd')
A, B, C, D, E = map(exprvar, 'ABCDE')
w, x, y, z = map(exprvar, 'wxyz')
W, X, Y, Z = map(exprvar, 'WXYZ')

In [57]:
# Insert your equation to the `insert_equation variable`
# V 1) Or(~a & ~c, a & b & c, a & c)
# V 2) Or(~Or(~x & ~y, z), z, x & y, w & z)
# V 3) Or(A & B & Or(C, D) & E, Or(C, D) & A & C)
# V 4) And(~A | C, ~A | ~C, A | B | ~C & D)
# V 5) Or(~a & ~b & c, a & d, b & ~d, c & ~d, a & ~c, ~a & ~d)

input_equation = Or(~a & ~b & c, a & d, b & ~d, c & ~d, a & ~c, ~a & ~d)
simplified_eq = input_equation.simplify().to_dnf()
# TODO: To LATEX
def expr_to_latex(k):
    print(type(k))
    # Check the expression's operator type
    if isinstance(k, boolalg.expr.AndOp):
        return f"({expr_to_latex(expr.args[0])} \\land {expr_to_latex(expr.args[1])})"
    elif isinstance(k, boolalg.expr.OrOp):
        return f"({expr_to_latex(expr.args[0])} \\lor {expr_to_latex(expr.args[1])})"
    elif isinstance(k, boolalg.expr.NotOp):
        return f"\\neg {expr_to_latex(expr.args[0])}"
    elif isinstance(k, boolalg.expr.XorOp):
        return f"({expr_to_latex(expr.args[0])} \\oplus {expr_to_latex(expr.args[1])})"
    else:
        return str(expr)



In [58]:
# Settings

predefined_colors = [PURPLE, MAROON, RED, ORANGE, GOLD, YELLOW, GREEN, TEAL, BLUE]

In [59]:
# Adds colors from the solved groups      

(row_vars, col_vars), input_matrix = kmap_animator.truth_table_to_kmap(input_equation)
mingroups = kmap_animator.create_min_groups(input_matrix)
mop_mingroup = []
color_picker = kmap_animator.color_selector(predefined_colors)
for mingroup_cluster in mingroups:
  selected_color = color_picker()
  for mingroup in mingroup_cluster:
    mop_mingroup.append({"coords": mingroup, "color": selected_color})

## Animation Output:

In [60]:
%%manim $_RV
# Animation manim magic to see
class Example(Scene):
    def construct(self):
        # Example matrix (you can replace this with any n x m matrix)
        matrix = input_matrix
        row_vars, col_vars
        answer = str(simplified_eq)

        # Call the function to generate the K-map
        kmap, labels = self.create_kmap(matrix, row_vars, col_vars)
        
        # Add a title
        title = Text("Karnaugh Map", font_size=24).to_edge(UP)
        self.play(Write(title))

        # Add the answer string, positioned above the K-map
        answer_text = Text(answer, font_size=24)
        self.play(Write(answer_text))

        # Wait a moment before moving the answer
        self.wait(1)
        self.play(FadeOut(answer_text))
        self.wait(1)
        # Add the K-map labels and grid to the scene
        self.play(Create(labels), Create(kmap))

        self.wait(1)
        
        # Define rectangles using top-left and bottom-right coordinates
        rectangles = mop_mingroup

        # Get the top-left corner of the K-map
        kmap_corner = kmap.get_corner(UL)  # UL = Upper Left in Manim
        kmap_bottom_right = kmap.get_corner(DR)  # DR = Down Right in Manim

        # Constants for cell dimensions
        cell_width = 1.0
        cell_height = 1.0
        cell_margin = 0
        corner_radius = 0.5
        dashes = 10


        # Add encompassing rectangle around the entire K-map
        kmap_width = kmap_bottom_right[0] - kmap_corner[0]
        kmap_height = kmap_corner[1] - kmap_bottom_right[1]
        kmap_rect = Rectangle(
            width=kmap_width,
            height=kmap_height,
            color=WHITE,
            fill_opacity=0.0,  # No fill, only border
        )
        kmap_rect.move_to(kmap.get_center())
        
        dashed_kmap_rect = DashedVMobject(kmap_rect, num_dashes=40)

        self.play(Create(kmap_rect))

        # Draw the rectangles with alpha transparency, clipped within the K-map boundaries
        for i, rect in enumerate(rectangles):
            # Unpack the given coordinates
            (coord1, coord2) = rect["coords"]
            color = rect["color"]

            # Calculate the most encompassing rectangle's corners
            top_left_row = min(coord1[1], coord2[1])
            top_left_col = min(coord1[0], coord2[0])
            bottom_right_row = max(coord1[1], coord2[1])
            bottom_right_col = max(coord1[0], coord2[0])

            # Calculate rectangle coordinates
            top_left_x = kmap_corner[0] + top_left_col * (cell_width)
            top_left_y = kmap_corner[1] - top_left_row * (cell_height)
            bottom_right_x = kmap_corner[0] + (bottom_right_col + 1) * (cell_width)
            bottom_right_y = kmap_corner[1] - (bottom_right_row + 1) * (cell_height)


            # Recalculate width, height, and center after clipping
            width = bottom_right_x - top_left_x + cell_margin 
            height = top_left_y - bottom_right_y + cell_margin 
            center_x = (top_left_x + bottom_right_x) / 2
            center_y = (top_left_y + bottom_right_y) / 2

            # Create the filled rectangle with transparency (alpha=0.2)
            rectangle = RoundedRectangle(
                corner_radius=min(corner_radius, width / 2, height / 2),
                width=width,
                height=height,
                color=color,
                stroke_opacity=0.3,
                fill_opacity=0.3  # Set transparency
            )
            
            rectangle.move_to([center_x, center_y, 0])
            clipped_rectangle = Intersection(rectangle, kmap_rect, color=color, fill_opacity = 0.1, stroke_opacity=0.3)
            # Add the rectangle to the scene
            self.play(Create(clipped_rectangle), run_time=0.5)
            

        self.wait(3)

    def create_kmap(self, matrix, row_vars, col_vars):
        """
        Create a flat Karnaugh Map representation for an n x m matrix with Gray code labels.
        """
        n = len(matrix)      # Number of rows
        m = len(matrix[0])   # Number of columns

        # Constants for layout
        cell_width = 1.0
        cell_height = 1.0
        cell_margin = 0

        # Group to hold all K-map elements
        kmap_group = VGroup()
        labels_group = VGroup()

        # Gray code generation for labels
        def gray_code(num_bits):
            return [bin(i ^ (i >> 1))[2:].zfill(num_bits) for i in range(2 ** num_bits)]

        row_labels = gray_code(int.bit_length(max(1, n - 1)))
        col_labels = gray_code(int.bit_length(max(1, m - 1)))

        get_color = lambda x: YELLOW if x == 1 else GRAY
        get_bold  = lambda x: NORMAL if x == 1 else NORMAL
        
        # Generate the grid of the K-map
        for i, row in enumerate(matrix):
            for j, value in enumerate(row):
                # Create the rectangle for the cell
                rect = Rectangle(width=cell_width, height=cell_height, color=GRAY,  stroke_opacity=0.1)
                rect.move_to(np.array([(j - m / 2) * (cell_width + cell_margin) + cell_width / 2,
                                    -(i - n / 2) * (cell_height + cell_margin) - cell_height / 2,
                                    0]))

                # Add the value as a text in the center of the cell
                text = Text(str(value), font_size=36, color=get_color(value), weight=get_bold(value))
                text.set_z_index(1)
                text.move_to(rect.get_center())

                # Group the rectangle and text together
                cell_group = VGroup(rect, text)

                # Add to the overall K-map group
                kmap_group.add(cell_group)

        def add_table_labels(idx, label, labelType: str):
            # Ensure label is a string before calling replace
            label = str(label).replace("\t", "")  # Convert label to string and remove tabs
            label_text = Text(label, font_size=24)  # Ensure no tabs in the text
            is_col = labelType == "COL"
            
            # Positioning for row and column labels
            if labelType == "ROW":
                label_text.next_to(kmap_group[0].get_left(), LEFT)
                label_text.shift(DOWN * idx * (cell_height + cell_margin))
            else:
                label_text.next_to(kmap_group[0].get_top(), UP)
                label_text.shift(RIGHT * idx * (cell_width + cell_margin))
            
            labels_group.add(label_text)

        
        

        # Add Gray code row and column labels
        for i, label in enumerate(row_labels):
            add_table_labels(i + len(row_vars), label, "ROW")

        for j, label in enumerate(col_labels):
            add_table_labels(j + len(col_vars), label, "COL")
        
        corner_label = "2"
        corner_label_text = Text(corner_label)
        corner_label_text.next_to(kmap_group[0].get_left(), LEFT, buff=0.5)
        corner_label_text.shift(UP)
        
        return kmap_group, labels_group

Manim Community v0.18.1